# <center> Music Generation personal project : predicting the following note with offset and duration

### by : Jerome Dauba

#Import

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

Mon Sep 13 12:35:01 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.63.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import random
from tqdm import tqdm

#Pytorch specific
import torch
from torch import nn

#Tensorflow specific
from keras.models import load_model
from keras.models import Sequential
from keras.layers import concatenate
from keras import Model
from keras.layers import Input, Dense, LSTM, Flatten, Dropout, Activation, CuDNNLSTM, Reshape
from keras.layers import LSTM, Dense, Bidirectional, Input,Dropout,BatchNormalization, CuDNNGRU, CuDNNLSTM
from keras import backend as K
from keras.layers import BatchNormalization as BatchNorm
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint
import tensorflow as tf

import os
from os import listdir
from os.path import isfile, join
import glob
from music21 import converter, instrument, note, chord, stream
from scipy.io import wavfile 
import IPython
from google.colab import files

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Collect Midi Files

In [ ]:
path_dataset = '/content/drive/MyDrive/Perso/AI Music Transition/A- Simple MIDI Version/Dataset/EDM MIDI/'
path_dir = '/content/drive/MyDrive/Perso/AI Music Transition/A- Simple MIDI Version/'
path_weights = '/content/drive/MyDrive/Perso/AI Music Transition/A- Simple MIDI Version/Weights/'
path_big_weights = '/content/drive/MyDrive/Perso/AI Music Transition/A- Simple MIDI Version/Weights/Bigger_NetworkV1/'
path_big_weights_enhanced = '/content/drive/MyDrive/Perso/AI Music Transition/A- Simple MIDI Version/Weights/Bigger_NetworkV1/Enhanced/'

In [ ]:
def get_notes():
	""" Get all the notes and chords from the midi files in the ./midi_songs directory """
	notes = []
	offsets = []
	durations = []

	for file in tqdm(glob.glob(join(path_dataset,"*.mid"))):
		midi = converter.parse(file)

		print("Parsing %s" % file)

		notes_to_parse = None

		try: # file has instrument parts
			s2 = instrument.partitionByInstrument(midi)
			notes_to_parse = s2.parts[0].recurse() 
		except: # file has notes in a flat structure
			notes_to_parse = midi.flat.notes
		
		
		offsetBase = 0
		for element in notes_to_parse:
			isNoteOrChord = False
			
			if isinstance(element, note.Note):
				notes.append(str(element.pitch))
				isNoteOrChord = True
			elif isinstance(element, chord.Chord):
				notes.append('.'.join(str(n) for n in element.normalOrder))
				isNoteOrChord = True
			
			if isNoteOrChord:
				#print("Offset Base = " + str(offsetBase))
				#print(element.offset - offsetBase)
				
				offsets.append(str(element.offset - offsetBase))
				
				#print(element.duration)
				#print(element.duration.quarterLength)
				
				durations.append(str(element.duration.quarterLength))
				
				isNoteOrChord = False
				offsetBase = element.offset
 
	return notes, offsets, durations

# Read and convert all Midi Files

In [ ]:
def prepare_sequences(notes, n_vocab):
	""" Prepare the sequences used by the Neural Network """
	sequence_length = 200

	# get all pitch names
	pitchnames = sorted(set(item for item in notes))

	 # create a dictionary to map pitches to integers
	note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

	network_input = []
	network_output = []

	# create input sequences and the corresponding outputs
	for i in range(0, len(notes) - sequence_length, 1):
		sequence_in = notes[i:i + sequence_length]
		sequence_out = notes[i + sequence_length]
		network_input.append([note_to_int[char] for char in sequence_in])
		network_output.append(note_to_int[sequence_out])

	n_patterns = len(network_input)

	# reshape the input into a format compatible with LSTM layers
	network_input = np.reshape(network_input, (n_patterns, sequence_length, 1))
	# normalize input
	network_input = network_input / float(n_vocab)

	network_output = np_utils.to_categorical(network_output)

	return (network_input, network_output)

# Training the predictive model

In [ ]:
def create_network(network_input_notes, n_vocab_notes, network_input_offsets, n_vocab_offsets, network_input_durations, n_vocab_durations):

  # Branch of the network that considers notes
  inputNotesLayer = Input(shape=(network_input_notes.shape[1], network_input_notes.shape[2]))
  inputNotes = Bidirectional(LSTM(
    256,
    input_shape=(network_input_notes.shape[1], network_input_notes.shape[2]),
    return_sequences=True
  ))(inputNotesLayer)
  inputNotes = Dropout(0.4)(inputNotes)

  # Branch of the network that considers note offset
  inputOffsetsLayer = Input(shape=(network_input_offsets.shape[1], network_input_offsets.shape[2]))
  inputOffsets = Bidirectional(LSTM(
    256,
    input_shape=(network_input_offsets.shape[1], network_input_offsets.shape[2]),
    return_sequences=True
  ))(inputOffsetsLayer)
  inputOffsets = Dropout(0.4)(inputOffsets)

  # Branch of the network that considers note duration
  inputDurationsLayer = Input(shape=(network_input_durations.shape[1], network_input_durations.shape[2]))
  inputDurations = Bidirectional(LSTM(
    256,
    input_shape=(network_input_durations.shape[1], network_input_durations.shape[2]),
    return_sequences=True
  ))(inputDurationsLayer)
  inputDurations = Dropout(0.4)(inputDurations)

  #Concatentate the three input networks together into one branch now
  inputs = concatenate([inputNotes, inputOffsets, inputDurations])

  # A LSTM to consider everything learnt from the three separate branches
  x = Bidirectional(LSTM(768, return_sequences=True))(inputs)
  x = Dropout(0.4)(x)
  x = Bidirectional(LSTM(512))(x)
  x = BatchNorm()(x)
  x = Dropout(0.4)(x)
  x = Dense(256, activation='relu')(x)

  #Time to split into three branches again...

  # Branch of the network that classifies the note
  outputNotes = Dense(256, activation='relu')(x)
  outputNotes = BatchNorm()(outputNotes)
  outputNotes = Dropout(0.3)(outputNotes)
  outputNotes = Dense(n_vocab_notes, activation='softmax', name="Note")(outputNotes)

  # Branch of the network that classifies the note offset
  outputOffsets = Dense(256, activation='relu')(x)
  outputOffsets = BatchNorm()(outputOffsets)
  outputOffsets = Dropout(0.3)(outputOffsets)
  outputOffsets = Dense(n_vocab_offsets, activation='softmax', name="Offset")(outputOffsets)

  # Branch of the network that classifies the note duration
  outputDurations = Dense(256, activation='relu')(x)
  outputDurations = BatchNorm()(outputDurations)
  outputDurations = Dropout(0.3)(outputDurations)
  outputDurations = Dense(n_vocab_durations, activation='softmax', name="Duration")(outputDurations)

  # Tell Keras what our inputs and outputs are 
  model = Model(inputs=[inputNotesLayer, inputOffsetsLayer, inputDurationsLayer], outputs=[outputNotes, outputOffsets, outputDurations])

  #Adam seems to be faster than RMSProp and learns better too 
  model.compile(loss='categorical_crossentropy', optimizer='adam',  metrics=["accuracy"])

  return model

In [ ]:
notes, offsets, durations = get_notes()

# Prepare notes 
n_vocab_notes = len(set(notes))
network_input_notes, network_output_notes = prepare_sequences(notes, n_vocab_notes)

# Prepare notes 
n_vocab_offsets = len(set(offsets))
network_input_offsets, network_output_offsets = prepare_sequences(offsets, n_vocab_offsets)

# Prepare notes 
n_vocab_durations = len(set(durations))
network_input_durations, network_output_durations = prepare_sequences(durations, n_vocab_durations)

  3%|▎         | 7/260 [00:00<00:04, 61.75it/s]

Parsing /content/drive/MyDrive/Perso/AI Music Transition/A- Simple MIDI Version/Dataset/EDM MIDI/2_E1_M2.mid
Parsing /content/drive/MyDrive/Perso/AI Music Transition/A- Simple MIDI Version/Dataset/EDM MIDI/3_E1_M3.mid
Parsing /content/drive/MyDrive/Perso/AI Music Transition/A- Simple MIDI Version/Dataset/EDM MIDI/4_E1_M4.mid
Parsing /content/drive/MyDrive/Perso/AI Music Transition/A- Simple MIDI Version/Dataset/EDM MIDI/5_E1_M5.mid
Parsing /content/drive/MyDrive/Perso/AI Music Transition/A- Simple MIDI Version/Dataset/EDM MIDI/6_E1_M6.mid
Parsing /content/drive/MyDrive/Perso/AI Music Transition/A- Simple MIDI Version/Dataset/EDM MIDI/7_E1_M7.mid
Parsing /content/drive/MyDrive/Perso/AI Music Transition/A- Simple MIDI Version/Dataset/EDM MIDI/8_E1_M8.mid
Parsing /content/drive/MyDrive/Perso/AI Music Transition/A- Simple MIDI Version/Dataset/EDM MIDI/9_E1_M9.mid
Parsing /content/drive/MyDrive/Perso/AI Music Transition/A- Simple MIDI Version/Dataset/EDM MIDI/10_E1_M10.mid
Parsing /content/

  7%|▋         | 18/260 [00:00<00:04, 53.37it/s]

Parsing /content/drive/MyDrive/Perso/AI Music Transition/A- Simple MIDI Version/Dataset/EDM MIDI/16_E2_M6.mid
Parsing /content/drive/MyDrive/Perso/AI Music Transition/A- Simple MIDI Version/Dataset/EDM MIDI/12_E2_M2.mid
Parsing /content/drive/MyDrive/Perso/AI Music Transition/A- Simple MIDI Version/Dataset/EDM MIDI/14_E2_M4.mid
Parsing /content/drive/MyDrive/Perso/AI Music Transition/A- Simple MIDI Version/Dataset/EDM MIDI/13_E2_M3.mid
Parsing /content/drive/MyDrive/Perso/AI Music Transition/A- Simple MIDI Version/Dataset/EDM MIDI/17_E2_M7.mid
Parsing /content/drive/MyDrive/Perso/AI Music Transition/A- Simple MIDI Version/Dataset/EDM MIDI/18_E2_M8.mid
Parsing /content/drive/MyDrive/Perso/AI Music Transition/A- Simple MIDI Version/Dataset/EDM MIDI/20_E2_M10.mid
Parsing /content/drive/MyDrive/Perso/AI Music Transition/A- Simple MIDI Version/Dataset/EDM MIDI/11_E2_M1.mid
Parsing /content/drive/MyDrive/Perso/AI Music Transition/A- Simple MIDI Version/Dataset/EDM MIDI/26_E3_M6.mid
Parsing /

 13%|█▎        | 34/260 [00:00<00:03, 58.65it/s]

...

Parsing /content/drive/MyDrive/Perso/AI Music Transition/A- Simple MIDI Version/Dataset/EDM MIDI/E27_M10.mid
Parsing /content/drive/MyDrive/Perso/AI Music Transition/A- Simple MIDI Version/Dataset/EDM MIDI/E27_M9.mid
Parsing /content/drive/MyDrive/Perso/AI Music Transition/A- Simple MIDI Version/Dataset/EDM MIDI/E27_M2.mid
Parsing /content/drive/MyDrive/Perso/AI Music Transition/A- Simple MIDI Version/Dataset/EDM MIDI/Episode28_Melody1.mid
Parsing /content/drive/MyDrive/Perso/AI Music Transition/A- Simple MIDI Version/Dataset/EDM MIDI/Episode28_Melody2.mid


 99%|█████████▉| 258/260 [00:11<00:00, 22.65it/s]

Parsing /content/drive/MyDrive/Perso/AI Music Transition/A- Simple MIDI Version/Dataset/EDM MIDI/Episode28_Melody3.mid
Parsing /content/drive/MyDrive/Perso/AI Music Transition/A- Simple MIDI Version/Dataset/EDM MIDI/Episode28_Melody4.mid
Parsing /content/drive/MyDrive/Perso/AI Music Transition/A- Simple MIDI Version/Dataset/EDM MIDI/Episode28_Melody5.mid
Parsing /content/drive/MyDrive/Perso/AI Music Transition/A- Simple MIDI Version/Dataset/EDM MIDI/Episode28_Melody6.mid
Parsing /content/drive/MyDrive/Perso/AI Music Transition/A- Simple MIDI Version/Dataset/EDM MIDI/Episode28_Melody7.mid
Parsing /content/drive/MyDrive/Perso/AI Music Transition/A- Simple MIDI Version/Dataset/EDM MIDI/Episode28_Melody8.mid
Parsing /content/drive/MyDrive/Perso/AI Music Transition/A- Simple MIDI Version/Dataset/EDM MIDI/Episode28_Melody9.mid


100%|██████████| 260/260 [00:11<00:00, 21.98it/s]

Parsing /content/drive/MyDrive/Perso/AI Music Transition/A- Simple MIDI Version/Dataset/EDM MIDI/Episode28_Melody10.mid


In [ ]:
model = create_network(network_input_notes, n_vocab_notes, network_input_offsets, n_vocab_offsets, network_input_durations, n_vocab_durations)

Checking if Keras is using GPU :

In [ ]:
from keras import backend as K
K._get_available_gpus()

['/device:GPU:0']

In [ ]:
model.load_weights(path_big_weights_enhanced+'ENHANCED_weights0.53.hdf5')

In [ ]:
# Training the model
filepath = "ENHANCED_weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"    
checkpoint = ModelCheckpoint(
    filepath, monitor='loss', 
    verbose=0,        
    save_best_only=True,        
    mode='min'
)    
callbacks_list = [checkpoint]
model.fit([network_input_notes, network_input_offsets, network_input_durations], [network_output_notes, network_output_offsets, network_output_durations], epochs=300, batch_size=64, callbacks=callbacks_list, verbose=1)

# Generating a new sequence from sampling the language model

To generate a new sequence from the language model, we simply give it as input a random sequence of duration ```sequence_length```and ask the trained network to predict the output (using ```model.predict```). 

The output of the network is a vector of probability of dimension $n_x$ which represents the probability of each note to be the next note of the melody given as input.

From this vector, we select the note which has the maximum probability.

We then concatenate this new note (its one-hot-encoding representation) at the end of the input sequence.
We finally remove the first element of the input sequence to keep its duration constant (```sequence_length```).

Instead of providing a random sequence as input, we rather randomly select one sequence out of the 2880 sequences used for training.
We denote it by ```pattern```.

# Loading model weights :

In [ ]:
model.load_weights(path_big_weights_enhanced+'ENHANCED_weights0.53.hdf5')

# Utils to predict notes, offsets and durations

In [ ]:
def generate_notes(model, network_input_notes, network_input_offsets, network_input_durations, notenames, offsetnames, durationames, n_vocab_notes, n_vocab_offsets, n_vocab_durations):
  """ Generate notes from the neural network based on a sequence of notes """
  # Alea threshold :
  threshold = 0.8
  # pick a random sequence from the input as a starting point for the prediction
  start = np.random.randint(0, len(network_input_notes)-1)
  start2 = np.random.randint(0, len(network_input_offsets)-1)
  start3 = np.random.randint(0, len(network_input_durations)-1)

  int_to_note = dict((number, note) for number, note in enumerate(notenames))
  #print(int_to_note)
  int_to_offset = dict((number, note) for number, note in enumerate(offsetnames))
  int_to_duration = dict((number, note) for number, note in enumerate(durationames))

  pattern = network_input_notes[start]
  #print("shape pattern:", pattern.shape)
  pattern2 = network_input_offsets[start2]
  pattern3 = network_input_durations[start3]

  prediction_output = []

	# generate notes or chords
  for note_index in tqdm(range(300)):
    alea = np.random.uniform()

    note_prediction_input = np.reshape(pattern, (1, len(pattern), 1))
    predictedNote = note_prediction_input[-1][-1][-1]
    #print(note_prediction_input.shape)

    #print(n_vocab_notes)
    #print(n_vocab_offsets)
    #print(n_vocab_durations)


    note_prediction_input = note_prediction_input / float(n_vocab_notes)

    offset_prediction_input = np.reshape(pattern2, (1, len(pattern2), 1))
    offset_prediction_input = offset_prediction_input / float(n_vocab_offsets)

    duration_prediction_input = np.reshape(pattern3, (1, len(pattern3), 1))
    duration_prediction_input = duration_prediction_input / float(n_vocab_durations)

    prediction = model.predict([note_prediction_input, offset_prediction_input, duration_prediction_input], verbose=0)
    #print('Pred OK')
    
    if alea < threshold :
      index = np.argmax(prediction[0])
      result = int_to_note[index]
      #print("\n", index)
    else:
      print("\n", "Choosing 2nd best")
      index = np.argsort(-prediction[0], axis=1)[:,1][0]
      #print("\n", index)
      result = int_to_note[index]


    offset = np.argmax(prediction[1])
    offset_result = int_to_offset[offset]
    #print("offset")
    #print(offset_result)

    duration = np.argmax(prediction[2])
    duration_result = int_to_duration[duration]
    #print("duration")
    #print(duration_result)

    #print("Next note: " + str(int_to_note[index]) + " - Duration: " + str(int_to_duration[duration]) + " - Offset: " + str(int_to_offset[offset]))


    prediction_output.append([result, offset_result, duration_result])

    pattern = np.vstack((pattern,index))
    pattern2 = np.vstack((pattern2,offset))
    pattern3 = np.vstack((pattern3,duration))

    pattern = pattern[1:len(pattern)]
    pattern2 = pattern2[1:len(pattern2)]
    pattern3 = pattern3[1:len(pattern3)]

  print("Prediction output length", len(prediction_output))
  return prediction_output

In [ ]:
def create_midi(prediction_output_all):
  """ convert the output from the prediction to notes and create a midi file
    from the notes """
  offset = 0
  output_notes = []
	
	#prediction_output = prediction_output_all
	
  offsets = []
  durations = []
  notes = []
	
  for x in prediction_output_all:
    notes = np.append(notes, x[0])
    try:
      offsets = np.append(offsets, float(x[1]))
    except:
      try:
        num, denom = x[1].split('/')
        x[1] = float(num)/float(denom)
        offsets = np_utils.append(offsets, float(x[1]))
      except:
        pass
      
    durations = np.append(durations, x[2])
	
	# create note and chord objects based on the values generated by the model
  x = 0 # this is the counter
  for pattern in notes:
    # pattern is a chord
    if ('.' in pattern) or pattern.isdigit():
      notes_in_chord = pattern.split('.')
      notes = []
      for current_note in notes_in_chord:
        new_note = note.Note(int(current_note))
        new_note.storedInstrument = instrument.Piano()
        notes.append(new_note)
      new_chord = chord.Chord(notes)
      
      try:
        new_chord.duration.quarterLength = float(durations[x])
      except:
        num, denom = durations[x].split('/')
        new_chord.duration.quarterLength = float(num)/float(denom)
      
      new_chord.offset = offset
      
      output_notes.append(new_chord)
    # pattern is a note
    else:
      new_note = note.Note(pattern)
      new_note.offset = offset
      new_note.storedInstrument = instrument.Piano()
      try:
        new_note.duration.quarterLength = float(durations[x])
      except:
        num, denom = durations[x].split('/')
        new_note.duration.quarterLength = float(num)/float(denom)
      
      output_notes.append(new_note)

		# increase offset each iteration so that notes do not stack
    try:
      offset += offsets[x]
    except:
      try:
        num, denom = offsets[x].split('/')
        offset += num/denom
      except:
        pass
				
    x = x+1

  midi_stream = stream.Stream(output_notes)

  midi_stream.write('midi', fp='test_output_enhanced.mid')

# Midi Generation

In [ ]:
notenames = sorted(set(item for item in notes))
offsetnames = sorted(set(item for item in offsets))
durationames = sorted(set(item for item in durations))

prediction_output = generate_notes(model, network_input_notes, network_input_offsets, network_input_durations, notenames, offsetnames, durationames, n_vocab_notes, n_vocab_offsets, n_vocab_durations)
create_midi(prediction_output)
files.download('/content/test_output_enhanced.mid')


  1%|          | 3/300 [00:00<00:30,  9.65it/s]


 Choosing 2nd best



  6%|▌         | 18/300 [00:01<00:28,  9.75it/s]


 Choosing 2nd best



 10%|▉         | 29/300 [00:03<00:28,  9.48it/s]

...


 Choosing 2nd best



100%|██████████| 300/300 [00:31<00:00,  9.43it/s]



 Choosing 2nd best
Prediction output length 300


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

To improve :


1.   Bidirectional LSTM
2.   LSTM: activation = ReLU
3.   Skip connections ?



Too much overfit ?